In [25]:
import json
import requests
import pprint
import tweepy
import info

import pandas as pd
import numpy as np

from importlib import reload
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

import matplotlib.pyplot as plt

In [17]:
with open(info.get_path()) as f:
    d = json.load(f)
    consumer_key = d["consumer_key"]
    consumer_secret = d["consumer_secret"]
    access_token = d["access_token"]
    access_token_secret = d['access_secret']

In [30]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [31]:
user = api.me()
query = "rick and morty"

In [118]:
data = [] # set your data list
text_set = set() # set your text list

In [32]:
searched_tweets = [status for status in tweepy.Cursor(api.search, q=query).items(5000)]

In [35]:
analyzer = SentimentIntensityAnalyzer()

In [119]:
for tweet in searched_tweets:    
    d=dict()
    text = tweet.text
    if text not in text_set:
        d["text"] = tweet.text # get tweet text
        d["date_created"] = tweet.created_at # get the date it was created
        # d["user"] = tweet.user.name # get user name
        blob = TextBlob(d["text"]) # create textblob object
        d["textblob_polarity"] = blob.sentiment.polarity # get polarity (positive/negative)
        d["textblob_subjectivity"] = blob.sentiment.subjectivity # get opinion/fact
        vader_dict = analyzer.polarity_scores(tweet.text)
        d.update(vader_dict)
        data.append(d)
        text_set.add(text)

In [120]:
df = pd.DataFrame(data)

In [121]:
df.shape

(1628, 8)

In [126]:
df["v_agrees_t"] = [1 if v*t > 0 else 0 for v, t in zip(df["compound"], df.textblob_polarity)]

In [127]:
df.head()

,compound,date_created,neg,neu,pos,text,textblob_polarity,textblob_subjectivity,v_agrees_t
0,0.0000,2018-11-06 21:37:56,0.0,1.000,0.000,RT @jovianjake: How is it possible that so man...,0.125,0.641667,0
1,0.6369,2018-11-06 21:36:58,0.0,0.704,0.296,I voted for #TeamRickAndMorty on @TheTylt—I lo...,0.500,0.600000,1
2,0.0000,2018-11-06 21:34:39,0.0,1.000,0.000,Today is a day for blunts and rick and morty e...,0.000,0.000000,0
3,0.0000,2018-11-06 21:34:19,0.0,1.000,0.000,Estas a ver Rick and morty ? https://t.co/7ahT...,0.000,0.000000,0
4,0.0000,2018-11-06 21:33:47,0.0,1.000,0.000,Rick and Morty: The Rickshank Rickdemption Dec...,-0.400,0.400000,0
